## 如果是在Google Colab上執行，請先執行以下程式碼

In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

## 如果在本機執行，需先設定好`conda vars`變數，以取得各金鑰

In [3]:
import os
line_channel_access_token = os.environ.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = os.environ.get('LINE_CHANNEL_SECRET')
port = 5051

In [4]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,    
)

from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                            TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5051
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2024 16:31:28] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[]}
BODY:  {"destination":"U08e238379070624ecd3a6ba91e449e19","events":[{"type":"message","message":{"type":"text","id":"538383881926869170","quoteToken":"2cLt3jgrqOzgxvGOODtVL1EdTHxjRmqZHbKRHEAc2R9GqIVOG96AMAs8r5ZgohWxs0B-THYdKu6yepYQj563kXbeu6eoB50FtcQvAGQdJhiH0GhEKSJlkRWuE-DxDd8sMTOu5Ef_mViw-8PWO9fFyg","text":"測試"},"webhookEventId":"01JEN8QEVCZ5PX2MT4H381C6F6","deliveryContext":{"isRedelivery":false},"timestamp":1733733104404,"source":{"type":"user","userId":"U0cbbc8f6ebbedcee3d7a054051db04a2"},"replyToken":"43c9cd613f8646f8a68db9a1e0042afb","mode":"active"}]}


127.0.0.1 - - [09/Dec/2024 16:31:44] "POST / HTTP/1.1" 200 -
